# Fashion Prof 👕
## Problem
The Out-of-Limbo Corp has produced a cloth series called "Off the Limbo" a thousad years ago. \
For halloween, they wanna redistributed the leftover (around 10 million clothes). \
To decrease the costs, they want an image classification model so that they can feed the cloth type to a machine which puts the clothes into boxes and pack them up.
## Data
Out-of-Limbo has manually classified 70,000 data to train the model. They stored it in a weird dataset called "Fashion MNIST" \
[Repo on GH](https://github.com/zalandoresearch/fashion-mnist)
## Evaluation
An accuracy of 0.8 will define the success of the model
> Cuz a mistake can be regarded as a Halloween Special
## Features
Just like this: `(image, label)`
Labels are in a one-hot encoded format (i.e. numerical). So, their association;

| Label | Description |
| --- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover |
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |

In [ ]:
# @title Importing the required libraries 📚
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

### Getting the Data Ready!

In [ ]:
# Importing the dataset
data, metadata = tfds.load("fashion_mnist", as_supervised=True, with_info=True)
data

In [ ]:
# Splitting the data into train and test
train_dataset, test_dataset = data["train"], data["test"]

In [ ]:
num_training_data = metadata.splits['train'].num_examples
num_testing_data = -metadata.splits['test'].num_examples

print(f"We have {num_training_data} data examples for training⚙️!")
print(f"We have {num_testing_data} data examples for testing🧪!")

In [ ]:
class_names = metadata.features["label"].names

In [ ]:
next(iter(train_dataset))

In [ ]:
# Every pixel ranges from 0-255. So normalising them...

def normalize(image, labels):
    image = tf.cast(image, tf.float32)   # Converting the pixel values to floats
    image /= 255   # Normalising
    return image, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

In [ ]:
# Caching them for faster processing...
train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

### EDA

In [ ]:
# Plotting a random image
image, label = next(iter(train_dataset))

image = image.numpy().reshape(28,28)

plt.figure(figsize=(3,3))
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.xlabel(class_names[label])
plt.xticks([])
plt.yticks([]);

`train_dataset.take(n)` spits out an iterable of that many values

In [ ]:
# Plotting 25 Images
for i, (image, label) in enumerate(train_dataset.take(25)):
    image = image.numpy().reshape(28,28)
    plt.subplot(5, 5, i+1)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.subplots_adjust(hspace=0.5)
    plt.xlabel(class_names[label])
    plt.grid(False)
    plt.xticks([])
    plt.yticks([]);

In [ ]:
# Checking out the image shape
image, label = next(iter(train_dataset))
image.shape

## Modelling

In [ ]:
def build_model():
    """
    Builds up a model and returns it
    """

    # Defining the model
    model = tf.keras.Sequential([
        # Convolution Layer 1
        tf.keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(28,28,1)),
        tf.keras.layers.MaxPool2D((2,2)),

        # Convolution Layer 2
        tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
        tf.keras.layers.MaxPool2D((2,2)),

        # Dense Network
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        metrics=["accuracy"],
        loss=tf.keras.losses.SparseCategoricalCrossentropy()
    )

    return model

In [ ]:
# Peaking into the architecture of the model!
smp_model = build_model()
smp_model.summary()

In [ ]:
BATCH_SIZE = 32 # @param {type:"slider", min:0, max:102, step:10}

In [ ]:
train_dataset = train_dataset.shuffle(buffer_size=num_training_data).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Instatiating the Generator Object with Property Ranges
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# You must need batches containing images.
augmented_img_batch = datagen.flow(train_dataset, save_prefix="test", save_format="jpeg")

In [ ]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Your cooool model has an accuracy score of {accuracy}")

## Getting Predictions

In [ ]:
for test_imgs, test_labels in test_dataset.take(1):
    print(test_imgs.shape)
    prediction = model.predict(test_imgs)
    print(prediction.shape)

In [ ]:
def plot_batch_preds():
    """
    Plots the predicted and the expected label along with the image!
    """
    for test_images, test_labels in test_dataset.take(1):
        test_images = test_images.numpy()
        test_labels = test_labels.numpy()
        pred_label_probs = model.predict(test_images)

        # Start plotting
        plt.figure(figsize=(18, 12))
        for i in range(BATCH_SIZE):
            # Choosing the subplot
            plt.subplot(8, 4, i+1)

            # Global settings
            plt.grid(False)
            plt.xticks([])
            plt.yticks([])

            # Plotting the image
            plt.imshow(test_images[i], cmap=plt.cm.binary)

            # Set the xlabel
            pred_label = np.argmax(pred_label_probs[i])
            if pred_label == test_labels[i]:
                color = "g"
                sign = "=="
            else:
                color = "r"
                sign = "!="

            caption = f"True: {class_names[test_labels[i]]} {sign} Pred: {class_names[pred_label]}"
            plt.xlabel(f"{caption} \n({np.max(pred_label_probs):.2f})", color=color)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_batch_preds()

In conclusion, the model is successful because it has an accuracy score which is greater than 0.8!